# Import important libraries

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

# Access website on browser

In [ ]:
url = "https://www.swiggy.com/city/bangalore"
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)

# Extracting Data using selenium and bs4

In [ ]:
data = []


count_page = 0

while (count_page<400):
    driver.execute_script('document.querySelectorAll(\'div[class^="RestaurantList__ShowMoreContainer"]\')[0].click()')
    count_page+=1


html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
view_link_div = soup.find('main',{'id':'seo-core-layout'}).\
findChildren('div',recursive=False)[8].find('div').find('div').find_all('a')

print(len(view_link_div))


res_links = list(lin['href'] for lin in view_link_div)

for res_link in res_links:
    try:
        driver.get(res_link)
        time.sleep(5)
        res_html = driver.page_source
        res_soup = BeautifulSoup(res_html, "html.parser")
        res_name = res_soup.\
        find('p',attrs={'class': lambda e: e.startswith('RestaurantNameAddress_name') if e else False}).get_text()
        rating = res_soup.\
        find('span',attrs={'class': lambda e: e.startswith('RestaurantRatings_avgRating') if e else False}).findChildren('span',recursive=False)[1].get_text()
        price_for_one_two = res_soup.find_all('li',attrs={'class': lambda e: e.startswith('RestaurantTimeCost_item') if e else False})[1].find('span').get_text()
        cusines = res_soup.\
        find('p',attrs={'class': lambda e: e.startswith('RestaurantNameAddress_cuisines') if e else False}).get_text()
        location = res_soup.\
        find('p',attrs={'class': lambda e: e.startswith('RestaurantNameAddress_area') if e else False}).get_text()
        delivery_review_number = res_soup.\
        find('span',attrs={'class': lambda e: e.startswith('RestaurantRatings_totalRatings') if e else False}).get_text()

        root = res_soup.find('div',{'id':'root'}).findChildren('div',recursive=False)[0].findChildren('div',recursive=False)[0].\
        find('div').find('div').findChildren('div',recursive=False)[1].findChildren('div',recursive=False)

        for i in range(4,len(root)-3):
            dish_category = root[i]['id'].replace('cid-','')
            dish_items = root[i].find('div').find('div').findChildren('div',recursive=False)

            for dish in dish_items:
                try:
                    dish_name = dish.find('h3',attrs={'class': lambda e: e.startswith('styles_itemNameText') if e else False}).get_text()
                    price = dish.find('span', class_ = 'rupee').get_text()
                    veg_status = 'Veg' if 'icon-Veg' in dish.find_all('i')[0].attrs['class'] else 'NonVeg'

                    data.append([res_link,res_name,rating,price_for_one_two,cusines,location,delivery_review_number,dish_category,dish_name,price,veg_status])
                except:
                    pass
    except:
        pass


5920


# convert extracted data into DataFrame

In [ ]:
df = pd.DataFrame(data,columns=['res_link','res_name','rating','price_for_one_two','cusines','location','delivery_review_number','dish_category','dish_name','price','veg_status'])

# add data into xlsx format

In [ ]:
df.to_excel(r'C:\Food_Delivery_Swiggy\swiggy_data_bangalore.xlsx')

## check Extracted data size

In [ ]:
df.shape

(456838, 11)

## check duplicated rows

In [ ]:
df.duplicated().sum()

348971

## check unique rows

In [ ]:
df.shape[0] - (df.duplicated().sum())

107867

## Delete duplicated rows and load data in new xlsx file

In [ ]:
df.shape

(456838, 11)

In [ ]:
dfr.shape

(107867, 11)

In [ ]:
dfr = df.copy()

In [ ]:
dfr.drop_duplicates(inplace=True)

In [ ]:
dfr.to_excel(r'C:\Food_Delivery_Swiggy\swiggy_data_main.xlsx')

In [ ]:
dfr.columns

Index(['res_link', 'res_name', 'rating', 'price_for_one_two', 'cusines',
       'location', 'delivery_review_number', 'dish_category', 'dish_name',
       'price', 'veg_status'],
      dtype='object')

In [ ]:
dfr['res_name'].nunique()

1390

In [ ]:
dfr.head()

,res_link,res_name,rating,price_for_one_two,cusines,location,delivery_review_number,dish_category,dish_name,price,veg_status
0,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,₹500 for two,"Biryani, Andhra",Jayanagar,10000,Recommended,Paneer 65,320,Veg
1,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,₹500 for two,"Biryani, Andhra",Jayanagar,10000,Recommended,Golden Baby Corn,300,Veg
2,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,₹500 for two,"Biryani, Andhra",Jayanagar,10000,Recommended,Chilly Chicken (Boneless),320,NonVeg
3,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,₹500 for two,"Biryani, Andhra",Jayanagar,10000,Recommended,Meghana Chicken 555,325,NonVeg
4,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,₹500 for two,"Biryani, Andhra",Jayanagar,10000,Recommended,Lemon Chicken,320,NonVeg


## clean "delivery_review_number" column

In [ ]:
import re

In [ ]:
dfr['delivery_review_number'] = dfr['delivery_review_number'].apply(lambda x : int(re.sub('[^0-9]','',x))*1000 if x.find('K')>=0 else int(re.sub('[^0-9]','',x)))

In [ ]:
dfr.delivery_review_number.unique()

array([10000,  1000,   500,   100,  5000,    50,    20,    10,     8,
           3,     4,     7,     5,     9,     2,     6], dtype=int64)

## clean "price_for_one_two" column and create two new columns "price_for_one" and "price_for_two" and drop the actual column

In [ ]:
dfr['price_for_one_two'].unique()

array(['₹500 for two', '₹400 for two', '₹200 for two', '₹350 for two',
       '₹250 for two', '₹300 for two', '₹100 for two', '₹150 for two',
       '₹270 for two', '₹750 for two', '₹299 for two', '₹450 for two',
       '₹1000 for two', '₹220 for two', '₹1400 for two', '₹550 for two',
       '₹900 for two', '₹600 for two', '₹800 for two', '₹120 for two',
       '₹99 for two', '₹190 for two', '₹349 for two', '₹80 for two',
       '₹1200 for two', '₹1300 for two', '₹384 for two', '₹700 for two',
       '₹650 for two', '₹160 for two', '₹240 for two', '₹50 for two',
       '₹499 for two', '₹170 for two', '₹339 for two', '₹198 for two',
       '₹60 for two', '₹850 for two', '₹180 for two', '₹130 for two',
       '₹1600 for two', '₹199 for two', '₹32 for two', '₹260 for two',
       '₹290 for two', '₹1100 for two', '₹230 for two', '₹149 for two',
       '₹280 for two', '₹375 for two', '₹1500 for two', '₹70 for two',
       '₹310 for two', '₹249 for two', '₹559 for two', '₹30 for two',
      

In [ ]:
dfr = dfr.assign( price_for_two = lambda x: dfr['price_for_one_two'].replace('[^0-9]','',regex=True).astype(int), price_for_one = lambda y: dfr['price_for_one_two'].replace('[^0-9]','',regex=True).astype(int)//2 )

In [ ]:
dfr.drop('price_for_one_two',axis=1,inplace=True)

In [ ]:
dfr.dtypes

res_link                  object
res_name                  object
rating                    object
cusines                   object
location                  object
delivery_review_number     int64
dish_category             object
dish_name                 object
price                     object
veg_status                object
price_for_two              int32
price_for_one              int32
dtype: object

## convert price column datatype into integer

In [ ]:
dfr['price'] = dfr['price'].astype(int)

In [ ]:
dfr.head(50)

,res_link,res_name,rating,cusines,location,delivery_review_number,dish_category,dish_name,price,veg_status,price_for_two,price_for_one
0,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,"Biryani, Andhra",Jayanagar,10000,Recommended,Paneer 65,320,Veg,500,250
1,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,"Biryani, Andhra",Jayanagar,10000,Recommended,Golden Baby Corn,300,Veg,500,250
2,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,"Biryani, Andhra",Jayanagar,10000,Recommended,Chilly Chicken (Boneless),320,NonVeg,500,250
3,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,"Biryani, Andhra",Jayanagar,10000,Recommended,Meghana Chicken 555,325,NonVeg,500,250
4,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,"Biryani, Andhra",Jayanagar,10000,Recommended,Lemon Chicken,320,NonVeg,500,250
5,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,"Biryani, Andhra",Jayanagar,10000,Recommended,Butter Chicken Curry,330,NonVeg,500,250
6,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,"Biryani, Andhra",Jayanagar,10000,Recommended,Paneer Butter Masala,320,Veg,500,250
7,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,"Biryani, Andhra",Jayanagar,10000,Recommended,Chicken Biryani,315,NonVeg,500,250
8,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,"Biryani, Andhra",Jayanagar,10000,Recommended,Chicken Boneless Biryani,325,NonVeg,500,250
9,https://www.swiggy.com/restaurants/meghana-foo...,Meghana Foods,4.4,"Biryani, Andhra",Jayanagar,10000,Recommended,Meghana Special Biryani,330,NonVeg,500,250


## convert 'rating' column datatype into float

In [ ]:
dfr['rating'].unique()

array([4.4, 4.5, 4.3, 4.2, 3.9, 4.6, 4.1, 3.8, 4.7, 3.7, 4. , 3.2, 3.5,
       3.4, 4.8, 3.3, 3.6, 2.8, 5. , 2.9, 2.7, 2.6, 3.1, 4.9, 2.5, 2.3,
       2.4, 2. , 1.7, 3. , 1.6, 2.2])

In [ ]:
dfr['rating'] = dfr['rating'].astype(float)

## check unique values of each column

In [ ]:
for i in dfr:
    print(i)
    print(dfr[i].unique())
    print(dfr[i].nunique(),'\n')

res_link
['https://www.swiggy.com/restaurants/meghana-foods-jayanagar-bangalore-655'
 'https://www.swiggy.com/restaurants/soul-rasa-akkithimanahalli-shanti-nagar-bangalore-614642'
 'https://www.swiggy.com/restaurants/great-indian-khichdi-by-eatfit-cunnigham-road-vasanth-nagar-bangalore-440123'
 ...
 'https://www.swiggy.com/restaurants/iced-sg-palya-tavarekere-bangalore-530959'
 'https://www.swiggy.com/restaurants/in-house-blend-ashok-nagar-central-bangalore-bangalore-770296'
 'https://www.swiggy.com/restaurants/naati-biryani-adda-nagarbhavi-vijayanagar-bangalore-736364']
1393 

res_name
['Meghana Foods' 'Soul Rasa' 'Great Indian Khichdi by EatFit' ... 'Iced'
 'In House Blend' 'Naati biryani adda']
1390 

rating
[4.4 4.5 4.3 4.2 3.9 4.6 4.1 3.8 4.7 3.7 4.  3.2 3.5 3.4 4.8 3.3 3.6 2.8
 5.  2.9 2.7 2.6 3.1 4.9 2.5 2.3 2.4 2.  1.7 3.  1.6 2.2]
32 

cusines
['Biryani, Andhra' 'Indian, Healthy Food' 'Home Food, Indian'
 'Andhra, Biryani' 'Bakery, Desserts' 'North Indian, Chinese'
 'American,